In [11]:
import numpy as np 
import pandas as pd 
import geopy
import folium
from wordcloud import WordCloud, STOPWORDS
import tweepy
import matplotlib.pyplot as plt
from textblob import TextBlob as tb
from googletrans import Translator
from unidecode import unidecode

# Scrapper

In [ ]:
API_key = ""
API_secret_key = ""
Acess_token = ""
Access_token_secret = ""

auth = tweepy.OAuthHandler(API_key, API_secret_key)
auth.set_access_token(Access_token, Access_token_secret)

api = tweepy.API(
    auth, 
    wait_on_rate_limit=True, 
    wait_on_rate_limit_notify=True, 
    retry_count=5, 
    retry_delay=10
)

In [ ]:
tweets = []
info = []
keyword = ['covid OR covid-19 OR corona OR coronavirus OR pandemic']

# tweet = token.search(q=keyword, count=10, result_type='mixed')

In [ ]:
for tweet in tweepy.Cursor(
    api.search, 
    q=keyword, 
    tweet_mode='extended', 
    rpp=1000, 
    result_type='mixed', 
    lang='en', 
    include_entities=True
).items(1000):
    if 'retweeted_status' in dir(tweet):
        aux = tweet.retweeted_status.full_text
    else:
        aux = tweet.full_text

    newtweet = aux.replace("\n", " ")

    tweets.append(newtweet)
    info.append(tweet)
    file = open('tweets_Keyword_covid_en_turma2.txt', 'a', -1, 'utf-8')
    file.write(newtweet+'\n')
    file.close()

In [ ]:
print(f'Total de tweets coletados {len(info)}.')

In [ ]:
tweets_df = pd.DataFrame(tweets, columns = ['Tweets'])

tweets_df['len'] = np.array([len(tweet) for tweet in tweets])
tweets_df['ID'] = np.array([tweet.id for tweet in info])
tweets_df['USER'] = np.array([tweet.user.screen_name for tweet in info])
tweets_df['userName'] = np.array([tweet.user.name for tweet in info])
tweets_df['userLocation'] = np.array([tweet.user.location for tweet in info])
tweets_df['Language'] = np.array([tweet.user.lang for tweet in info])
tweets_df['Date'] = np.array([tweet.created_at for tweet in info])
tweets_df['Source'] = np.array([tweet.source for tweet in info])
tweets_df['Likes'] = np.array([tweet.favorite_count for tweet in info])
tweets_df['Retweets'] = np.array([tweet.retweet_count for tweet in info])
tweets_df['Geo'] = np.array([tweet.geo for tweet in info])
tweets_df['Coordinates'] = np.array([tweet.coordinates for tweet in info])
tweets_df['Place'] = np.array([tweet.place for tweet in info])

tweets_df.to_csv('./tweets_Keyword_covid_en_Turma2.csv')

In [ ]:
likes_max = np.max(tweets_df['Likes'])

likes = tweets_df[tweets_df.Likes == likes_max].index[0]

print(f'O tweet com mais curtidas é: {tweets_df['Tweets'][likes]} com {likes_max} curtidas.')

In [ ]:
print(np.sum(tweets_df['Likes'] == likes_max))

In [ ]:
retweet_max = np.max(tweets_df['Retweets'])

retweet = tweets_df[tweets_df.Retweets == retweet_max].index[0]

print(f'O tweet com mais retweet é {tweets_df['Tweets'][retweet]} com {retweet_max} retweets')

# Análise de Sentimento

In [ ]:
sources = []

for source in tweets_df['Source']:
    if source not in sources:
        sources.append(source)

percent = np.zeros(len(sources))

for source in tweets_df['Source']:
    for index in range(len(sources)):
        if source == sources[index]:
            percent[index] += 1
            
            pass

In [ ]:
sourceDF = pd.DataFrame({'source': percent}, index=sources)

sourceDF

In [ ]:
sources_sorted = sourceDF.sort_values('source', ascending=True)
ax = sources_sorted.source.plot(kind='barh', color='#A52A2A')
ax.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

In [ ]:
sources_sorted = sourceDF.sort_values('source', ascending=True)
ax = sources_sorted.source.plot(kind='barh', color='#B8860B')
ax.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))


In [ ]:
analysis = None
polarities = []

for tweet in tweets_df['Tweets']:
    analysis = tb(tweet)

    polarity = analysis.sentiment.polarity

    polarities.append(polarity)

In [ ]:
## Alternative
for tweet in tweets_df['Tweets']:
    polarity = tb(tweet).sentiment.polarity

    polarities.append(polarity)

In [ ]:
print('Vetor de polaridade: ', polarity)

In [ ]:
print(f'Para as palavras: {keyword}\nA média de sentimento é: {str(np.mean(polarities))}')

In [ ]:
positive, negative, neutral = 0, 0, 0

for polarity in polarities:
    if polarity > 0:
        positive += 1
    elif polarity < 0:
        negative += 1
    else:
        neutral += 1

pos_pct = positive * 100/len(polarities)
neg_pct = negative * 100/len(polarities)
neu_pct = neutral * 100/len(polarities)

sentiment_label = ['Positivos', 'Negativos', 'Neutros']
percents = [pos_pct, neg_pct, neu_pct]

In [ ]:
pie_chart = pd.Series(percents, index=sentiment_label, name='')

In [ ]:
pie_chart.plot.pie(fontsize=12, autopct='%.2f', figsize=(5, 5), labels=pie_chart.index, title='Análise de Sentimento Tweets')

In [ ]:
# Usando matplotlib
patchets, text = plt.pie(percents, startangle=90)
plt.legend(patches, sentiments, loc='best')
plt.tight_layout();

# Mapa localizador de Tweets

In [ ]:
## libs folium e geo
locator = Nominatim(user_agent='TweeterSentiments')
exemplo = locator.geocode('Belo Horizonte')
exemplo

In [ ]:
latitude = []
longitude = []

for user_location in tweets_df['User Location']:
    try:
        location = geolocator.geocode(user_location)
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except:
        continue

coordenadas = np.column_stack((latitude, longitude))

mapa = folium.Map(zoom_start=3)
mapa.add_child(plugins.HeatMap(coordenadas))
mapa.save('Mapa_Calor_covid_en_Turma2.html')
mapa

# Nuvem de palavras // wordcloud

In [ ]:
# from wordcloud import WordCloud, STOPWORDS
#import warnings

In [ ]:
words = ' '.join(tweets_df['Tweets'])

In [ ]:
words_clean = " ".join([
    word for word in words.split()
        if 'https' not in word 
            and not word.startswith('@') 
            and not word.startswith('#')
            and word != 'RT'
])

In [ ]:
warnings.simplefilter('ignore')
wc = WordCloud(
    min_font_size=10,
    max_font_size=300,
    background_color='white',
    mode='RGB',
    width=2000,
    height=1000,
    normalize_plurals=True
).generate(words_clean)

plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.savefig('covid_cloud_pt_Turma2.png', dpi=300);

In [ ]:
stopwords = set(STOPWORDS)
new_words = []

with open("stopwords_pt.txt", 'r') as f:
    [new_words.append(word) for line in f for word in line.split()]

new_stopwords = stopwords.union(new_words)

words = ' '.join(tweets_df['Tweets'])

words_clean = " ".join([
    word for word in words.split()
        if 'https' not in word 
            and not word.startswith('@') 
            and not word.startswith('#')
            and word != 'RT'
])

In [ ]:
warnings.simplefilter('ignore')
wc = WordCloud(
    min_font_size=10,
    max_font_size=300,
    background_color='white',
    mode='RGB',
    width=2000,
    height=1000,
    normalize_plurals=True
).generate(words_clean)

plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.savefig('covid_cloud_pt_Turma2.png', dpi=300);